<a href="https://colab.research.google.com/github/tosiki1202/GenAI-app/blob/main/finalTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 必要なライブラリのインストール

In [1]:
!pip install -q transformers accelerate langchain langchain-core langchain-community langchainhub langchain-mcp-adapters sentencepiece bitsandbytes
!pip install chromadb -q

# 2.モデルのロード

In [2]:
!pip install transformers accelerate bitsandbytes -q

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define the BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config, # Use quantization_config instead of load_in_4bit and torch_dtype
    dtype=torch.bfloat16 # Use dtype instead of torch_dtype
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 3. csvファイルの整形

In [3]:
import pandas as pd
import os
import sys

# --- 設定 ---
INPUT_FILE = "games_march2025_cleaned.csv"  # 入力ファイル名
OUTPUT_FILE = "steam_games_filtered_final.csv" # 最終出力ファイル名
CUTOFF_DATE = '2015-01-01' # この日付以前の行を破棄します (リリース日がこれより古いゲームは除外)
MIN_POSITIVE_REVIEWS = 7000 # 閾値を設定
ENCODING = "utf-8"

# 🔥 抽出したい列名をリストで指定してください 🔥
# 日付フィルタに必要な列と、保持したい列のみを指定
COLUMNS_TO_KEEP = [
    'release_date',
    'name',
    'genres',
    'price',
    'short_description',
    'detailed_description',
    'positive', # positive 列を保持
    'negative',
    'developers',
    'reviews',
    'about_the_game',
    'supported_languages'
]

# --- 処理 ---

def process_csv_for_date_and_positive_filter():
    """CSVファイルを読み込み、日付フィルタとpositiveレビュー数フィルタを適用して保存する関数。"""

    if not os.path.exists(INPUT_FILE):
        print(f"❌ エラー: 入力ファイル '{INPUT_FILE}' が見つかりません。ファイルがアップロードされているか確認してください。", file=sys.stderr)
        return

    print(f"1. ファイル '{INPUT_FILE}' を読み込み、必要な列を抽出します...")
    try:
        # 1. 必要な列だけをメモリに読み込む (usecolsによるメモリ効率化)
        # 'release_date'列は必ず含める必要があります
        cols_to_use = [col for col in COLUMNS_TO_KEEP if col in pd.read_csv(INPUT_FILE, encoding=ENCODING, nrows=1).columns]
        if 'release_date' not in cols_to_use:
             print(f"❌ エラー: CSVファイルに 'release_date' 列が見つかりません。処理を中断します。", file=sys.stderr)
             return
        # positive 列が必要な場合は追加
        if 'positive' not in cols_to_use and 'positive' in COLUMNS_TO_KEEP:
             cols_to_use.append('positive')


        df = pd.read_csv(INPUT_FILE, encoding=ENCODING, usecols=cols_to_use)

        # 処理開始時の元の行数を記録
        original_rows_count = len(df)
        current_rows_count = original_rows_count

        # 2. 日付列の変換とフィルタリング
        if 'release_date' in df.columns:
            print(f"2. 日付フィルター '{CUTOFF_DATE}' を適用します (この日付以降のデータのみを残します)...")
            # 'release_date'列のNaNを除外し、型を変換
            df.dropna(subset=['release_date'], inplace=True)
            df['release_date'] = pd.to_datetime(df['release_date'])

            cutoff_datetime = pd.to_datetime(CUTOFF_DATE)

            # リリース日がCUTOFF_DATEよりも新しい行を抽出
            df_filtered_date = df[df['release_date'] >= cutoff_datetime].copy() # >= に変更して2022年を含むようにする
            rows_dropped_by_date = current_rows_count - len(df_filtered_date)
            current_rows_count = len(df_filtered_date)
            print(f"   -> 日付フィルタで {rows_dropped_by_date} 行を削除しました。")
            print(f"   -> 日付フィルタ後の行数: {current_rows_count} 行")
        else:
            print("⚠️ 'release_date' 列が見つかりません。日付フィルタリングはスキップします。")
            df_filtered_date = df.copy() # 日付フィルタがない場合はdf全体をコピー


        # 3. positive レビュー数でのフィルタリング

        if 'positive' in df_filtered_date.columns:
             print(f"3. positive レビュー数フィルター ({MIN_POSITIVE_REVIEWS}件以上) を適用します...")
             # positive 列を数値型に変換し、NaNを削除してからフィルタリング
             df_filtered_date['positive'] = pd.to_numeric(df_filtered_date['positive'], errors='coerce')
             df_filtered_date.dropna(subset=['positive'], inplace=True)

             df_filtered_final = df_filtered_date[df_filtered_date['positive'] >= MIN_POSITIVE_REVIEWS].copy()
             rows_dropped_by_positive = current_rows_count - len(df_filtered_final)
             current_rows_count = len(df_filtered_final)
             print(f"   -> positive レビュー数フィルタで {rows_dropped_by_positive} 行を削除しました。")
             print(f"   -> フィルタリング後の最終行数: {current_rows_count} 行")
        else:
             print("⚠️ 'positive' 列が見つかりません。positive レビュー数フィルタリングはスキップします。")
             df_filtered_final = df_filtered_date.copy() # positive 列がない場合は日付フィルタ後のDataFrameをそのまま使用


        # 4. 最終的なCSVファイルとして保存
        print(f"4. 最終的なDataFrameを '{OUTPUT_FILE}' として保存します...")
        # COLUMNS_TO_KEEPで指定した列をそのまま保存
        df_filtered_final.to_csv(OUTPUT_FILE, index=False, encoding=ENCODING)

        # 保存後のファイルサイズを確認
        final_size_bytes = os.path.getsize(OUTPUT_FILE)
        final_size_mb = final_size_bytes / (1024 * 1024)
        print(f"   -> ファイル '{OUTPUT_FILE}' を保存しました (サイズ: {final_size_mb:.2f} MB).")
        print("✅ CSV処理完了。")

    except FileNotFoundError:
        print(f"❌ エラー: ファイル '{INPUT_FILE}' が見つかりません。", file=sys.stderr)
    except Exception as e:
        print(f"❌ 処理中にエラーが発生しました: {e}", file=sys.stderr)

# 関数を実行
process_csv_for_date_and_positive_filter()

1. ファイル 'games_march2025_cleaned.csv' を読み込み、必要な列を抽出します...
2. 日付フィルター '2015-01-01' を適用します (この日付以降のデータのみを残します)...
   -> 日付フィルタで 2771 行を削除しました。
   -> 日付フィルタ後の行数: 86847 行
3. positive レビュー数フィルター (7000件以上) を適用します...
   -> positive レビュー数フィルタで 85465 行を削除しました。
   -> フィルタリング後の最終行数: 1382 行
4. 最終的なDataFrameを 'steam_games_filtered_final.csv' として保存します...
   -> ファイル 'steam_games_filtered_final.csv' を保存しました (サイズ: 6.17 MB).
✅ CSV処理完了。


# 4. LangChain 用ラッパーの定義

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    top_p=0.95,
    temperature=0.5
)

Device set to use cuda:0


In [5]:
from langchain.llms.base import LLM
from typing import Optional, List
from pydantic import Field, model_validator # Pydantic関連をインポート

class LocalLLM(LLM):
    # Pydantic v2 の設定: extra='allow' で追加フィールドを許可
    model_config = {'extra': 'allow'}

    system_prompt: Optional[str] = Field(default=None) # system_promptを明示的にフィールドとして定義

    def __init__(self, system_prompt: str = None, **kwargs):
        super().__init__(**kwargs)
        self.system_prompt = system_prompt

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # プロンプトテンプレートを汎用的なものに変更
        if self.system_prompt:
            # システムプロンプトとユーザープロンプトを結合
            full_prompt = f"<s>[INST] <<SYS>>\n{self.system_prompt}\n<</SYS>>\n\n{prompt} [/INST]"
        else:
            full_prompt = f"<s>[INST] {prompt} [/INST]"


        output = llm_pipeline(full_prompt, max_new_tokens=1024, do_sample=True)[0]["generated_text"]

        # モデルの出力から入力プロンプト部分と [/INST] を削除してAssistantの応答のみを抽出
        # ELYZAモデルの出力形式に合わせて調整
        assistant_prefix = "[/INST]"
        if assistant_prefix in output:
            return output.split(assistant_prefix, 1)[1].strip()
        else:
            # もし[/INST]が出力に含まれない場合は、出力全体を返すか、調整が必要
            return output.strip()


    @property
    def _llm_type(self) -> str:
        return "local-llm" # タイプ名を汎用的に変更

# システムプロンプトを指定してLLMを初期化
# ここにあなたのシステムプロンプトを記入してください
system_prompt = "必ず日本語で出力してください．前提条件のみから出力してください．ポジティブなレビューの数と価格，ゲームの説明を必ず行なってください．ユーザの購買意欲を掻き立てるように説明してください．"
llm = LocalLLM(system_prompt=system_prompt)

print("LocalLLM with system prompt is ready.")

LocalLLM with system prompt is ready.


#5. RAG チェーンを構築

In [6]:
import torch
import os # デバッグ用にosをインポート
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA

# ----------------------------------------------------
# 【Step 0: 環境設定とLLMの準備】
# ----------------------------------------------------

# GPU が利用可能であることを確認し、デバイスを決定
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ Running on {device}: GPU will be used for embedding.")
else:
    device = "cpu"
    print(f"⚠️ Running on {device}. GPU not found, using CPU for embedding.")

try:
    llm # llm変数が定義されているか確認
except NameError:
    llm = None
    print("⚠️ LLM (llm変数) は定義されていません。RetrievalQAチェーンは構築できません。")


# ----------------------------------------------------
# 【Step 1: ドキュメントの読み込みと分割】
# ----------------------------------------------------

# 指定されたCSVファイルを読み込む (ファイル名はカレントディレクトリに配置されている前提)
csv_file_path = "steam_games_filtered_final.csv"
if not os.path.exists(csv_file_path):
    print(f"❌ Error: CSV file not found at {csv_file_path}. Please check the file path.")
    exit()

print(f"1. Loading document from {csv_file_path}...")
loader = CSVLoader(csv_file_path, encoding="utf-8")
docs = loader.load()

# テキスト分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
chunks = text_splitter.split_documents(docs)
print(f"   -> Document split into {len(chunks)} chunks.")


# ----------------------------------------------------
# 【Step 2: データのベクトル化とChroma VectorStoreの構築】
# ----------------------------------------------------

print(f"2. Initializing SentenceTransformer on {device}...")
embedding = SentenceTransformerEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": device},
    # バッチサイズを増やす (デフォルトは32)
    encode_kwargs={'batch_size': 64} # 適宜調整してください
)

print("3. Creating Chroma VectorStore (Embedding phase started, GPU may be busy)...")
vectorstore = Chroma.from_documents(chunks, embedding=embedding)
print("   -> VectorStore built successfully.")


# ----------------------------------------------------
# 【Step 3: Retrieval QA チェーン作成】
# ----------------------------------------------------

if llm is not None:
    # Retrieval QA チェーンの作成
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever()
    )
    print("4. RetrievalQA chain ready. You can now use qa.run('Your question here')")
else:
    print("4. RetrievalQA chain skipped because LLM (llm variable) is not defined.")

✅ Running on cuda: GPU will be used for embedding.
1. Loading document from steam_games_filtered_final.csv...
   -> Document split into 1450 chunks.
2. Initializing SentenceTransformer on cuda...


/tmp/ipython-input-3510208420.py:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(


3. Creating Chroma VectorStore (Embedding phase started, GPU may be busy)...
   -> VectorStore built successfully.
4. RetrievalQA chain ready. You can now use qa.run('Your question here')


# 6.出力

In [7]:
import torch

query = "日本語対応で，マルチプレイ可能な評価の高いゲームを教えて" #QueryはRAGに読み込ませた内容に応じて各自で変更。RAGに読み込ませた知識に関する問い合わせをする。

# 実行
try:
    response = qa.run(query)
    print("回答:", response)
except RuntimeError as e:
    print("CUDAメモリエラーが発生しました。対処案:")
    print("- モデルサイズを縮小")
    print("- トークン数を制限")
    print("- 埋め込みモデルをCPUに固定")
    print(f"詳細: {e}")

/tmp/ipython-input-3925162713.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


回答: <think>
まず、ユーザーが日本語で、マルチプレイ可能な評価の高いゲームを知りたいと理解しました。まず、提供された4つのゲームの情報を見ると、Name: 魔女的夜宴、Price: 34.99、 supported_languages: ['Japanese', 'Simplified Chinese', 'Traditional Chinese']、positive: 8085、negative: 110。このゲームは、3人でプレイ可能で、ユーザーが評価が很高く、positively 8085件、negative 110件です。この(game)がマルチプレイ可能性があり、ユーザーが strongly recommend する可能性があります。

次に、Name: Sanfu、Price: 6.59、 supported_languages: ['Simplified Chinese']、positive: 13481、negative: 2821。このゲームは、2人でプレイ可能で、positively 13481件、negative 2821件。ユーザーが strongly recommend する可能性がありますが、support languageはSimplified Chineseのみで、ユーザーが日本語を支持しているため、ダウンロードが容易です。

Name: OMORI、Price: 19.99、 supported_languages: ['English', 'Japanese', 'Simplified Chinese', 'Korean']、positive: 75941、negative: 2144。このゲームは、マルチプレイ可能性があり、positively 75941件、negative 2144件。ユーザーが strongly recommend する可能性があります。支持 language は多元一 oats,ユーザーが日本語を支持しているため、ダウンロードが容易です。

Name: Senren＊Banka、Price: 19.94、 supported_languages: ['Japanese', 'Simplified Chinese', 'Traditional Chinese', 'English']、positi